In [1]:
import math
from sklearn import neighbors, svm
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

In [2]:
train_dir = "train"
test_dir = "test"

trainX = []
trainY = []

In [3]:
for class_dir in os.listdir(train_dir):

    for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
        
        image = face_recognition.load_image_file(img_path)
        face_bounding_boxes = face_recognition.face_locations(image)

        if len(face_bounding_boxes) == 1:
            trainX.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
            trainY.append(class_dir)

In [4]:
print(len(trainX))
print(len(trainY))

1588
1588


In [5]:
def trainKNN(model_save_path=None, n_neighbors=None, knn_algo='ball_tree'):
    
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(trainX))))

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(trainX, trainY)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [6]:
print("Training KNN classifier...")
knn_clf = trainKNN(model_save_path="trained_knn_model.clf", n_neighbors=2)
print("Training complete!")

Training KNN classifier...
Training complete!


In [8]:
def predictKNN(X_img_path, knn_clf=None):
    
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    if len(X_face_locations) != 1:
        return "unkown"

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)
    
    return knn_clf.predict(faces_encodings)[0]

In [9]:
total = 0
nrCorecte = 0
for class_dir in os.listdir("test"):
    for image_file in image_files_in_folder(os.path.join("test", class_dir)):

        full_file_path = image_file

        name = predictKNN(full_file_path, knn_clf=knn_clf)
        
        total = total + 1
        if name == class_dir:
            nrCorecte = nrCorecte + 1
        else:
            print("Corect: " + class_dir + "; Prezis: " + name)

print("Corecte KNN: " + str(nrCorecte))
print("Totale KNN: " + str(total))
print("Acc KNN: " + str(nrCorecte/total))

Corect: Katherine Langford; Prezis: Alexandra Daddario
Corect: Britt Robertson; Prezis: Karen Gillan
Corecte KNN: 398
Totale KNN: 400
Acc KNN: 0.995


In [10]:
def trainSVM(model_save_path=None):

    svm_clf = svm.SVC(kernel='linear', C = 1.0)
    svm_clf.fit(trainX, trainY)
    
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(svm_clf, f)

    return svm_clf

In [11]:
print("Training SVM classifier...")
svm_clf = trainSVM(model_save_path="trained_svm_model.clf")
print("Training complete!")

Training SVM classifier...
Training complete!


In [12]:
def predictSVM(X_img_path):
    
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    if len(X_face_locations) != 1:
        return []

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    return svm_clf.predict(faces_encodings)[0]

In [13]:
total = 0
nrCorecte = 0
for class_dir in os.listdir("test"):
    for image_file in image_files_in_folder(os.path.join("test", class_dir)):

        full_file_path = image_file

        name = predictSVM(full_file_path)
        
        total = total + 1
        if name == class_dir:
            nrCorecte = nrCorecte + 1
        else:
            print("Corect: " + class_dir + "; Prezis: " + name)

print("Corecte SVM: " + str(nrCorecte))
print("Totale SVM: " + str(total))
print("Acc SVM: " + str(nrCorecte/total))

Corect: Katherine Langford; Prezis: Alexandra Daddario
Corecte SVM: 399
Totale SVM: 400
Acc SVM: 0.9975
